# COVID19-related literature SQL database

In this notebook, we create a relational database dump of a set of COVID19-related publication datasets. These include:

* CORD19: https://pages.semanticscholar.org/coronavirus-research
* Dimensions: https://docs.google.com/spreadsheets/d/1-kTZJZ1GAhJ2m4GAIhw1ZdlgO46JpvX0ZQa232VWRmw/edit#gid=2034285255
* WHO: https://www.who.int/emergencies/diseases/novel-coronavirus-2019/global-research-on-novel-coronavirus-2019-ncov

In [1]:
# magics, warnings and inports

%load_ext autoreload
%autoreload 2
import warnings; warnings.simplefilter('ignore')

import os, random, codecs, json
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import pymysql
from sqlalchemy import create_engine
from sqlalchemy import Integer,String,Boolean,DateTime

#### Load datasets

In [2]:
# point here to the versions of the datasets you want to use
dimensions_filename = "datasets_input/Dimensions_04_04_2020.csv"
who_filename = "datasets_input/WHO_04_04_2020.csv"
cord19_folder = "datasets_input/CORD19_2020_04_04"

df_dimensions = pd.read_csv(dimensions_filename, dtype=str)
df_who = pd.read_csv(who_filename, dtype=str)
df_cord = pd.read_csv(os.path.join(cord19_folder,"metadata.csv"), dtype=str)

FileNotFoundError: [Errno 2] File datasets_input/CORD19_2020_03_28/metadata.csv does not exist: 'datasets_input/CORD19_2020_03_28/metadata.csv'

In [3]:
df_cord.shape

(47298, 18)

### Prepare dataframes for ingestion

#### Clean-up data frames

##### Dimensions

In [4]:
df_dimensions.head()

,Date added,Publication ID,DOI,PMID,PMCID,Title,Abstract,Source title,Source UID,Publisher,...,Research Organizations - standardized,GRID IDs,City of Research organization,Country of Research organization,Funder,UIDs of supporting grants,Times cited,Altmetric,Source Linkout,Dimensions URL
0,2020-03-31,pub.1125964842,10.33448/rsd-v9i5.3151,NaN,NaN,A tutoria na educação à distância em tempos de...,This work aims to analyze tutoring in Distance...,Research Society and Development,jour.1314528,"Research, Society and Development",...,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,https://app.dimensions.ai/details/publication/...
1,2020-03-31,pub.1125962316,10.21203/rs.3.rs-18247/v1,NaN,NaN,Severe Acute Respiratory Syndrome Coronavirus ...,"<title xmlns=""http://www.ncbi.nlm.nih.gov/JATS...",Research Square,jour.1380788,Research Square,...,NaN,NaN,NaN,NaN,NaN,NaN,0,1,NaN,https://app.dimensions.ai/details/publication/...
2,2020-03-31,pub.1125963084,10.24018/ejmed.2020.2.2.194,NaN,NaN,Clinical Characterization of COVID-19,The world is on the brink of a pandemic due to...,European Journal of Medical and Health Sciences,jour.1367413,European Open Access Publishing (Europa Publis...,...,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,https://app.dimensions.ai/details/publication/...
3,2020-03-31,pub.1125969724,10.21203/rs.3.rs-19398/v1,NaN,NaN,Clinical retrospective analysis of 70 discharg...,"<title xmlns=""http://www.ncbi.nlm.nih.gov/JATS...",Research Square,jour.1380788,Research Square,...,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,https://app.dimensions.ai/details/publication/...
4,2020-03-31,pub.1125962317,10.21203/rs.3.rs-18255/v1,NaN,NaN,Clinical characteristics of 10 children with C...,"<title xmlns=""http://www.ncbi.nlm.nih.gov/JATS...",Research Square,jour.1380788,Research Square,...,NaN,NaN,NaN,NaN,NaN,NaN,0,1,NaN,https://app.dimensions.ai/details/publication/...


In [5]:
df_dimensions.columns

Index(['Date added', 'Publication ID', 'DOI', 'PMID', 'PMCID', 'Title',
       'Abstract', 'Source title', 'Source UID', 'Publisher', 'MeSH terms',
       'Publication Date', 'PubYear', 'Volume', 'Issue', 'Pagination',
       'Open Access', 'Publication Type', 'Authors', 'Corresponding Authors',
       'Authors Affiliations', 'Research Organizations - standardized',
       'GRID IDs', 'City of Research organization',
       'Country of Research organization', 'Funder',
       'UIDs of supporting grants', 'Times cited', 'Altmetric',
       'Source Linkout', 'Dimensions URL'],
      dtype='object')

In [6]:
df_dimensions.drop(columns=['Date added', 'Publisher', 'Authors', 'Corresponding Authors',
       'Authors Affiliations', 'Research Organizations - standardized',
       'GRID IDs', 'City of Research organization',
       'Country of Research organization', 'Funder',
       'UIDs of supporting grants', 'Times cited', 'Altmetric',
       'Source Linkout'], inplace=True)

In [7]:
df_dimensions.columns

Index(['Publication ID', 'DOI', 'PMID', 'PMCID', 'Title', 'Abstract',
       'Source title', 'Source UID', 'MeSH terms', 'Publication Date',
       'PubYear', 'Volume', 'Issue', 'Pagination', 'Open Access',
       'Publication Type', 'Dimensions URL'],
      dtype='object')

In [8]:
df_dimensions.rename(columns={'Publication ID':'publication_id', 'DOI':'doi', 'PMID':'pmid', 'PMCID':'pmcid', 'Title':'title', 'Abstract':'abstract',
       'Source title':'journal', 'Source UID':'source_uid', 'MeSH terms':'mesh_terms', 'Publication Date':'publication_date',
       'PubYear':'publication_year', 'Volume':'volume', 'Issue':'issue', 'Pagination':'pages', 'Open Access':'open_access',
       'Publication Type':'publication_type', 'Dimensions URL':'dimensions_url'}, inplace=True)

In [9]:
def get_year(date):
    if len(date)>3 and date[:4].isdigit():
        return date[:4]
    return ""

month_to_number = {"Jan":"1","Feb":"2","Mar":"3","Apr":"4","May":"5","Jun":"6","Jul":"7","Aug":"8","Sep":"9","Oct":"10","Nov":"11","Dec":"12"}

def get_month(date):
    if len(date)>6:
        if "-" in date and date.split("-")[1].isdigit():
            return str(int(date.split("-")[1]))
        else:
            try:
                return month_to_number[date.split()[1]]
            except:
                return ""
    return ""

In [10]:
df_dimensions["publication_year"] = df_dimensions["publication_year"].apply(get_year)
df_dimensions["publication_month"] = df_dimensions["publication_date"].apply(get_month)

In [11]:
df_dimensions.drop(columns="publication_date", inplace=True)
df_dimensions = df_dimensions.fillna('')

In [12]:
df_dimensions.head()

,publication_id,doi,pmid,pmcid,title,abstract,journal,source_uid,mesh_terms,publication_year,volume,issue,pages,open_access,publication_type,dimensions_url,publication_month
0,pub.1125964842,10.33448/rsd-v9i5.3151,,,A tutoria na educação à distância em tempos de...,This work aims to analyze tutoring in Distance...,Research Society and Development,jour.1314528,,2020,9,5,30953151,Closed,article,https://app.dimensions.ai/details/publication/...,3
1,pub.1125962316,10.21203/rs.3.rs-18247/v1,,,Severe Acute Respiratory Syndrome Coronavirus ...,"<title xmlns=""http://www.ncbi.nlm.nih.gov/JATS...",Research Square,jour.1380788,,2020,,,,"All OA; Green, Submitted",preprint,https://app.dimensions.ai/details/publication/...,3
2,pub.1125963084,10.24018/ejmed.2020.2.2.194,,,Clinical Characterization of COVID-19,The world is on the brink of a pandemic due to...,European Journal of Medical and Health Sciences,jour.1367413,,2020,2,2,,Closed,article,https://app.dimensions.ai/details/publication/...,3
3,pub.1125969724,10.21203/rs.3.rs-19398/v1,,,Clinical retrospective analysis of 70 discharg...,"<title xmlns=""http://www.ncbi.nlm.nih.gov/JATS...",Research Square,jour.1380788,,2020,,,,"All OA; Green, Submitted",preprint,https://app.dimensions.ai/details/publication/...,3
4,pub.1125962317,10.21203/rs.3.rs-18255/v1,,,Clinical characteristics of 10 children with C...,"<title xmlns=""http://www.ncbi.nlm.nih.gov/JATS...",Research Square,jour.1380788,,2020,,,,"All OA; Green, Submitted",preprint,https://app.dimensions.ai/details/publication/...,3


In [13]:
df_dimensions[df_dimensions.doi==""].shape

(193, 17)

##### WHO

In [14]:
df_who.head()

,Title,Authors,Abstract,Published Year,Published Month,Journal,Volume,Issue,Pages,Accession Number,DOI,Ref,Covidence #,Study,Notes,Tags
0,SARS-CoV-2 is not detectable in the vaginal fl...,"Qiu, Lin; Liu, Xia; Xiao, Meng; Xie, Jing; Cao...",Background Severe acute respiratory syndrome c...,2020,NaN,Clinical Infectious Diseases,NaN,NaN,NaN,NaN,10.1093/cid/ciaa375,26513,#27487,Qiu 2020,NaN,NaN
1,A New Application of Social Impact in Social M...,"Pulido, Cristina M.; Ruiz-Eugenio, Laura; Redo...",One of the challenges today is to face fake ne...,2020,NaN,International Journal of Environmental Researc...,17,7,2430-2430,NaN,10.3390/IJERPH17072430,26499,#27413,Pulido 2020,NaN,NaN
2,A medicinal chemistry perspective of drug repo...,"Pillaiyar, Thanigaimalai; Meenakshisundaram, S...",Drug repurposing is a strategy consisting of f...,2020,NaN,European Journal of Medicinal Chemistry,NaN,NaN,112275-112275,NaN,https://doi.org/10.1016/j.ejmech.2020.112275,26744,#27869,Pillaiyar 2020,NaN,NaN
3,Plano estadual de contingência para o enfrenta...,"Piauí . Secretaria de, Estado",O coronavírus (nCoV) faz parte de uma família ...,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26447,#27815,Piauí 2020,NaN,NaN
4,The Plague: Human resilience and the collectiv...,"Peters, Michael A.",NaN,2020,NaN,Educational Philosophy and Theory,NaN,NaN,1-4,NaN,10.1080/00131857.2020.1745921,27114,#27905,Peters 2020,NaN,NaN


In [15]:
df_who.columns

Index(['Title', 'Authors', 'Abstract', 'Published Year', 'Published Month',
       'Journal', 'Volume', 'Issue', 'Pages', 'Accession Number', 'DOI', 'Ref',
       'Covidence #', 'Study', 'Notes', 'Tags'],
      dtype='object')

In [16]:
df_who.drop(columns="Authors", inplace=True)

In [17]:
df_who.rename(columns={'Title':'title', 'Abstract':'abstract', 'Published Year':'publication_year', 'Published Month':'publication_month',
       'Journal':'journal', 'Volume':'volume', 'Issue':'issue', 'Pages':'pages', 'Accession Number':'accession_number', 'DOI':'doi', 'Ref':'ref',
       'Covidence #':'covidence', 'Study':'study', 'Notes':'notes', 'Tags':'tags'}, inplace=True)

In [18]:
df_who["pmid"] = ""
df_who["pmcid"] = ""
df_who = df_who.fillna('')

In [19]:
df_who.head()

,title,abstract,publication_year,publication_month,journal,volume,issue,pages,accession_number,doi,ref,covidence,study,notes,tags,pmid,pmcid
0,SARS-CoV-2 is not detectable in the vaginal fl...,Background Severe acute respiratory syndrome c...,2020,,Clinical Infectious Diseases,,,,,10.1093/cid/ciaa375,26513,#27487,Qiu 2020,,,,
1,A New Application of Social Impact in Social M...,One of the challenges today is to face fake ne...,2020,,International Journal of Environmental Researc...,17,7,2430-2430,,10.3390/IJERPH17072430,26499,#27413,Pulido 2020,,,,
2,A medicinal chemistry perspective of drug repo...,Drug repurposing is a strategy consisting of f...,2020,,European Journal of Medicinal Chemistry,,,112275-112275,,https://doi.org/10.1016/j.ejmech.2020.112275,26744,#27869,Pillaiyar 2020,,,,
3,Plano estadual de contingência para o enfrenta...,O coronavírus (nCoV) faz parte de uma família ...,2020,,,,,,,,26447,#27815,Piauí 2020,,,,
4,The Plague: Human resilience and the collectiv...,,2020,,Educational Philosophy and Theory,,,1-4,,10.1080/00131857.2020.1745921,27114,#27905,Peters 2020,,,,


In [20]:
df_who[df_who.doi==""].shape

(556, 17)

##### CORD19

In [21]:
df_cord.head()

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,Microsoft Academic Paper ID,WHO #Covidence,has_pdf_parse,has_pmc_xml_parse,full_text_file,url
0,8q5ondtn,NaN,Elsevier,Intrauterine virus infections and congenital h...,10.1016/0002-8703(72)90077-4,NaN,4361535,els-covid,Abstract The etiologic basis for the vast majo...,1972-12-31,"Overall, James C.",American Heart Journal,NaN,NaN,False,False,custom_license,https://doi.org/10.1016/0002-8703(72)90077-4
1,pzfd0e50,NaN,Elsevier,Coronaviruses in Balkan nephritis,10.1016/0002-8703(80)90355-5,NaN,6243850,els-covid,NaN,1980-03-31,"Georgescu, Leonida; Diosi, Peter; Buţiu, Ioan;...",American Heart Journal,NaN,NaN,False,False,custom_license,https://doi.org/10.1016/0002-8703(80)90355-5
2,22bka3gi,NaN,Elsevier,Cigarette smoking and coronary heart disease: ...,10.1016/0002-8703(80)90356-7,NaN,7355701,els-covid,NaN,1980-03-31,"Friedman, Gary D",American Heart Journal,NaN,NaN,False,False,custom_license,https://doi.org/10.1016/0002-8703(80)90356-7
3,zp9k1k3z,aecbc613ebdab36753235197ffb4f35734b5ca63,Elsevier,Clinical and immunologic studies in identical ...,10.1016/0002-9343(73)90176-9,NaN,4579077,els-covid,"Abstract Middle-aged female identical twins, o...",1973-08-31,"Brunner, Carolyn M.; Horwitz, David A.; Shann,...",The American Journal of Medicine,NaN,NaN,True,False,custom_license,https://doi.org/10.1016/0002-9343(73)90176-9
4,cjuzul89,NaN,Elsevier,Epidemiology of community-acquired respiratory...,10.1016/0002-9343(85)90361-4,NaN,4014285,els-covid,Abstract Upper respiratory tract infections ar...,1985-06-28,"Garibaldi, Richard A.",The American Journal of Medicine,NaN,NaN,False,False,custom_license,https://doi.org/10.1016/0002-9343(85)90361-4


In [22]:
# NEW columns (for now, we drop)
df_cord.drop(columns=["cord_uid","url","has_pmc_xml_parse"],inplace=True)

In [23]:
df_cord.columns

Index(['sha', 'source_x', 'title', 'doi', 'pmcid', 'pubmed_id', 'license',
       'abstract', 'publish_time', 'authors', 'journal',
       'Microsoft Academic Paper ID', 'WHO #Covidence', 'has_pdf_parse',
       'full_text_file'],
      dtype='object')

In [24]:
df_cord.drop(columns='authors', inplace=True)
df_cord = df_cord.fillna('')

In [25]:
df_cord.rename(columns={'source_x':'source', 'pubmed_id': 'pmid',
       'Microsoft Academic Paper ID': 'ms_academic_id', 'WHO #Covidence': 'who_covidence', 'has_pdf_parse':'has_full_text'}, inplace=True)

In [26]:
df_cord["publication_year"] = df_cord["publish_time"].apply(get_year)
df_cord["publication_month"] = df_cord["publish_time"].apply(get_month)

In [27]:
df_cord.drop(columns='publish_time', inplace=True)

In [28]:
df_cord['pages'] = ""
df_cord['volume'] = ""
df_cord['issue'] = ""

In [29]:
df_cord.head()

,sha,source,title,doi,pmcid,pmid,license,abstract,journal,ms_academic_id,who_covidence,has_full_text,full_text_file,publication_year,publication_month,pages,volume,issue
0,,Elsevier,Intrauterine virus infections and congenital h...,10.1016/0002-8703(72)90077-4,,4361535,els-covid,Abstract The etiologic basis for the vast majo...,American Heart Journal,,,False,custom_license,1972,12,,,
1,,Elsevier,Coronaviruses in Balkan nephritis,10.1016/0002-8703(80)90355-5,,6243850,els-covid,,American Heart Journal,,,False,custom_license,1980,3,,,
2,,Elsevier,Cigarette smoking and coronary heart disease: ...,10.1016/0002-8703(80)90356-7,,7355701,els-covid,,American Heart Journal,,,False,custom_license,1980,3,,,
3,aecbc613ebdab36753235197ffb4f35734b5ca63,Elsevier,Clinical and immunologic studies in identical ...,10.1016/0002-9343(73)90176-9,,4579077,els-covid,"Abstract Middle-aged female identical twins, o...",The American Journal of Medicine,,,True,custom_license,1973,8,,,
4,,Elsevier,Epidemiology of community-acquired respiratory...,10.1016/0002-9343(85)90361-4,,4014285,els-covid,Abstract Upper respiratory tract infections ar...,The American Journal of Medicine,,,False,custom_license,1985,6,,,


In [79]:
df_cord[(df_cord.doi=="") & ((df_cord.sha!="") | (df_cord.pmid!="") | (df_cord.pmcid!=""))].shape

(3043, 18)

In [31]:
df_dimensions.shape

(6659, 17)

In [32]:
df_who.shape

(4190, 17)

In [71]:
df_cord.shape

(47298, 18)

### Prepare tables

In [34]:
# the main table: pub

In [102]:
pub_table_columns = ['title','abstract','publication_year','publication_month','journal','volume','issue','pages','doi','pmid','pmcid']

df_pub = df_dimensions[pub_table_columns].append(df_who[pub_table_columns], ignore_index = True)

In [103]:
df_pub = df_pub[pub_table_columns].append(df_cord[pub_table_columns], ignore_index=True)

In [104]:
df_pub.shape

(58147, 11)

In [105]:
df_pub[(df_pub.doi=="") & (df_pub.pmid=="") & (df_pub.pmcid=="")].shape

(1011, 11)

In [106]:
# check to have at least one valid identifier per publication
# we drop publications which do not: hopefully, they will be equipped with an identifier in future releases

df_pub = df_pub[~((df_pub.doi=="") & (df_pub.pmid=="") & (df_pub.pmcid==""))]

In [107]:
# drop duplicates, first on dois then pmids then pmcids. We need this to keep empty values!
df_tmp = df_pub[df_pub.doi==""]
df_pub = df_pub[df_pub.doi!=""].groupby('doi').first()
df_pub.reset_index(inplace=True)
df_tmp2 = df_tmp[df_tmp.pmid==""]
df_pub2 = df_tmp[df_tmp.pmid!=""].groupby('pmid').first()
df_pub2.reset_index(inplace=True)
df_pub3 = df_tmp2[df_tmp2.pmcid!=""].groupby('pmcid').first()
df_pub3.reset_index(inplace=True)

In [108]:
df_pub = pd.concat([df_pub,df_pub2,df_pub3])

In [109]:
# add PK and reset index
df_pub.reset_index(drop=True,inplace=True)
df_pub["pub_id"] = df_pub.index.values

In [110]:
df_pub.shape

(51843, 12)

In [111]:
df_pub.tail()

,doi,title,abstract,publication_year,publication_month,journal,volume,issue,pages,pmid,pmcid,pub_id
51838,,Daily Situation Report on Coronavirus disease ...,,2020,3,Arch Acad Emerg Med,,,,,PMC7085938,51838
51839,,Incidentally Diagnosed COVID-19 Infection in T...,,2020,3,Arch Acad Emerg Med,,,,,PMC7092921,51839
51840,,Risks of Novel Coronavirus Disease (COVID-19) ...,INTRODUCTION: The outbreak of the new Coronavi...,2020,3,Arch Acad Emerg Med,,,,,PMC7092922,51840
51841,,The COVID-19 Intubation and Ventilation Pathwa...,,2020,3,Arch Acad Emerg Med,,,,,PMC7096723,51841
51842,,Prevalence of Underlying Diseases in Hospitali...,"INTRODUCTION: In the beginning of 2020, an une...",2020,3,Arch Acad Emerg Med,,,,,PMC7096724,51842


In [112]:
df_pub.dtypes

doi                  object
title                object
abstract             object
publication_year     object
publication_month    object
journal              object
volume               object
issue                object
pages                object
pmid                 object
pmcid                object
pub_id                int64
dtype: object

In [125]:
# create other tables via joins

df_datasource = pd.DataFrame.from_dict({"source":["CORD19","Dimensions","WHO"],"url":["https://pages.semanticscholar.org/coronavirus-research","https://docs.google.com/spreadsheets/d/1-kTZJZ1GAhJ2m4GAIhw1ZdlgO46JpvX0ZQa232VWRmw/edit#gid=2034285255",
"https://www.who.int/emergencies/diseases/novel-coronavirus-2019/global-research-on-novel-coronavirus-2019-ncov"]})
df_cord_metadata = df_cord[['source','license','full_text_file','ms_academic_id','who_covidence','doi','pmid','pmcid','sha']]
df_who_metadata = df_who[['accession_number', 'doi', 'ref',
       'covidence', 'study', 'notes', 'tags', 'pmid', 'pmcid']]
df_dimensions_metadata = df_dimensions[['publication_id', 'doi', 'pmid', 'pmcid', 'source_uid', 'mesh_terms',
       'open_access', 'publication_type', 'dimensions_url']]

In [126]:
df_datasource.head()

,source,url
0,CORD19,https://pages.semanticscholar.org/coronavirus-...
1,Dimensions,https://docs.google.com/spreadsheets/d/1-kTZJZ...
2,WHO,https://www.who.int/emergencies/diseases/novel...


In [127]:
# CORD19 metadata

In [128]:
#We need this to keep empty values!

df_tmp = df_cord_metadata[df_cord_metadata.doi==""]
df_cord_metadata1 = pd.merge(df_cord_metadata[df_cord_metadata.doi!=""], df_pub[['pub_id','doi']],  how='inner', left_on=['doi'], right_on=['doi'])
df_tmp2 = df_tmp[df_tmp.pmid==""]
df_cord_metadata2 = pd.merge(df_tmp[df_tmp.pmid!=""], df_pub[['pub_id','pmid']],  how='inner', left_on=['pmid'], right_on=['pmid'])
df_cord_metadata3 = pd.merge(df_tmp2[df_tmp2.pmcid!=""], df_pub[['pub_id','pmcid']],  how='inner', left_on=['pmcid'], right_on=['pmcid'])

In [129]:
df_cord_metadata1 = df_cord_metadata1.groupby("doi").first()
df_cord_metadata1.reset_index(inplace=True)
df_cord_metadata2 = df_cord_metadata2.groupby("pmid").first()
df_cord_metadata2.reset_index(inplace=True)
df_cord_metadata3 = df_cord_metadata3.groupby("pmcid").first()
df_cord_metadata3.reset_index(inplace=True)

In [130]:
df_cord_metadata = pd.concat([df_cord_metadata1,df_cord_metadata2,df_cord_metadata3])

In [131]:
df_cord_metadata.shape

(46994, 10)

In [132]:
# read full texts in
folders = ['biorxiv_medrxiv/biorxiv_medrxiv/pdf_json','comm_use_subset/comm_use_subset/pdf_json','custom_license/custom_license/pdf_json','noncomm_use_subset/noncomm_use_subset/pdf_json']
shas = list()
full_texts = list()

for folder in folders:
    for root, dirs, files in os.walk(os.path.join(cord19_folder,folder)):
        for file in tqdm(files):
            if ".json" in file: # read
                data = json.loads(codecs.open(os.path.join(root,file)).read())
                sha = data["paper_id"]
                full_text = "\n".join(section["text"] for section in data["body_text"])
                shas.append(sha)
                full_texts.append(full_text)

In [133]:
df_cord_fulltext = pd.DataFrame.from_dict({"sha":shas,"full_text":full_texts})

In [134]:
df_cord_metadata = pd.merge(df_cord_metadata, df_cord_fulltext,  how='left', left_on=['sha'], right_on=['sha'])
df_cord_metadata = df_cord_metadata.fillna('')
df_cord_metadata.rename(columns={"id":"pub_id"},inplace=True)

In [135]:
df_cord_metadata.head()

,doi,source,license,full_text_file,ms_academic_id,who_covidence,pmid,pmcid,sha,pub_id,full_text
0,0.1126/science.abb7331,WHO,unk,,,#8463,,,,0,
1,10.0376/cma.j.issn.0376-2491.2020.0002,WHO,unk,,3003451419,#615,32036640,,,1,
2,10.1001/archinte.168.22.2489,PMC,unk,,,,19064834,PMC2783624,,2,
3,10.1001/jama.2010.675,PMC,unk,,,,20501927,PMC2968755,,3,
4,10.1001/jama.2014.2116,PMC,unk,,,,24566924,PMC6689404,,4,


In [136]:
# WHO and Dimensions metadata

In [137]:
df_tmp = df_who_metadata[df_who_metadata.doi==""]
df_who_metadata1 = pd.merge(df_who_metadata[df_who_metadata.doi!=""], df_pub[['pub_id','doi']],  how='inner', left_on=['doi'], right_on=['doi'])
df_tmp2 = df_tmp[df_tmp.pmid==""]
df_who_metadata2 = pd.merge(df_tmp[df_tmp.pmid!=""], df_pub[['pub_id','pmid']],  how='inner', left_on=['pmid'], right_on=['pmid'])
df_who_metadata3 = pd.merge(df_tmp2[df_tmp2.pmcid!=""], df_pub[['pub_id','pmcid']],  how='inner', left_on=['pmcid'], right_on=['pmcid'])

In [138]:
df_who_metadata1 = df_who_metadata1.groupby("doi").first()
df_who_metadata1.reset_index(inplace=True)
df_who_metadata2 = df_who_metadata2.groupby("pmid").first()
df_who_metadata2.reset_index(inplace=True)
df_who_metadata3 = df_who_metadata3.groupby("pmcid").first()
df_who_metadata3.reset_index(inplace=True)

In [139]:
df_who_metadata = pd.concat([df_who_metadata1,df_who_metadata2,df_who_metadata3])

In [140]:
df_who_metadata.shape

(3478, 10)

In [141]:
df_who_metadata.rename(columns={"id":"pub_id"},inplace=True)

In [142]:
df_tmp = df_dimensions_metadata[df_dimensions_metadata.doi==""]
df_dimensions_metadata1 = pd.merge(df_dimensions_metadata[df_dimensions_metadata.doi!=""], df_pub[['pub_id','doi']],  how='inner', left_on=['doi'], right_on=['doi'])
df_tmp2 = df_tmp[df_tmp.pmid==""]
df_dimensions_metadata2 = pd.merge(df_tmp[df_tmp.pmid!=""], df_pub[['pub_id','pmid']],  how='inner', left_on=['pmid'], right_on=['pmid'])
df_dimensions_metadata3 = pd.merge(df_tmp2[df_tmp2.pmcid!=""], df_pub[['pub_id','pmcid']],  how='inner', left_on=['pmcid'], right_on=['pmcid'])

In [143]:
df_dimensions_metadata1 = df_dimensions_metadata1.groupby("doi").first()
df_dimensions_metadata1.reset_index(inplace=True)
df_dimensions_metadata2 = df_dimensions_metadata2.groupby("pmid").first()
df_dimensions_metadata2.reset_index(inplace=True)
df_dimensions_metadata3 = df_dimensions_metadata3.groupby("pmcid").first()
df_dimensions_metadata3.reset_index(inplace=True)

In [144]:
df_dimensions_metadata = pd.concat([df_dimensions_metadata1,df_dimensions_metadata2,df_dimensions_metadata3])

In [145]:
df_dimensions_metadata.shape

(6338, 10)

In [146]:
df_dimensions_metadata.rename(columns={"id":"pub_id"},inplace=True)

In [147]:
# Create datasource tables

In [148]:
cord_source_id = df_datasource[df_datasource.source=="CORD19"].index.values[0]
who_source_id = df_datasource[df_datasource.source=="WHO"].index.values[0]
dimensions_source_id = df_datasource[df_datasource.source=="Dimensions"].index.values[0]

In [149]:
df_cord_metadata["source_id"] = cord_source_id
df_who_metadata["source_id"] = who_source_id
df_dimensions_metadata["source_id"] = dimensions_source_id

In [150]:
df_pub_to_datasource = df_cord_metadata[["pub_id","source_id"]]
df_pub_to_datasource = df_pub_to_datasource.append(df_who_metadata[["pub_id","source_id"]],ignore_index=True)
df_pub_to_datasource = df_pub_to_datasource.append(df_dimensions_metadata[["pub_id","source_id"]],ignore_index=True)

In [151]:
df_pub_to_datasource.drop_duplicates(inplace=True)
df_pub_to_datasource.rename(columns={"source_id":"datasource_id"},inplace=True)

In [152]:
df_pub_to_datasource.shape

(56809, 2)

In [153]:
df_pub_to_datasource[df_pub_to_datasource.pub_id==22787]

,pub_id,datasource_id
21846,22787,0


In [154]:
# remove unnecessary columns
df_cord_metadata.drop(columns=['doi','pmid','pmcid','source_id'],inplace=True)
df_who_metadata.drop(columns=['doi','pmid','pmcid','source_id'],inplace=True)
df_dimensions_metadata.drop(columns=['doi','pmid','pmcid','source_id'],inplace=True)

In [155]:
# reset all indexes which will become PKs
df_cord_metadata.reset_index(drop=True,inplace=True)
df_who_metadata.reset_index(drop=True,inplace=True)
df_dimensions_metadata.reset_index(drop=True,inplace=True)
df_datasource.reset_index(drop=True,inplace=True)
df_cord_metadata["cord19_metadata_id"] = df_cord_metadata.index.values
df_who_metadata["who_metadata_id"] = df_who_metadata.index.values
df_dimensions_metadata["dimensions_metadata_id"] = df_dimensions_metadata.index.values
df_datasource["datasource_metadata_id"] = df_datasource.index.values

In [156]:
# make numeric where needed
df_pub["publication_year"] = pd.to_numeric(df_pub["publication_year"])
df_pub["publication_month"] = pd.to_numeric(df_pub["publication_month"])
df_pub["pmid"] = pd.to_numeric(df_pub["pmid"])

In [157]:
# add timestamp
df_pub["timestamp"] = pd.Timestamp.now()

In [78]:
# clean-up text (optional)
replaces = [""]

def clean_up(txt):
    for r in replaces:
        txt = txt.replace(r,"")
    return txt.encode('utf8', 'ignore').decode('utf8')
df_pub["abstract"] = [clean_up(a) for a in df_pub["abstract"].values]

In [158]:
df_pub.head()

,doi,title,abstract,publication_year,publication_month,journal,volume,issue,pages,pmid,pmcid,pub_id,timestamp
0,0.1126/science.abb7331,‘A ticking time bomb’: Scientists worry about ...,"CAPE TOWN, SOUTH AFRICA—Late on Sunday evening...",2020.0,NaN,Science,,,,NaN,,0,2020-04-04 07:55:51.892454
1,10.0376/cma.j.issn.0376-2491.2020.0002,[Ten hot issues of breast cancer under the nov...,,2020.0,2.0,Chinese medical journal,100,0,e002,32036640.0,,1,2020-04-04 07:55:51.892454
2,10.1001/archinte.168.22.2489,Another Piece of the Puzzle: Human Metapneumov...,BACKGROUND: Each winter respiratory viruses ac...,2008.0,12.0,Archives of Internal Medicine,,,,19064834.0,PMC2783624,2,2020-04-04 07:55:51.892454
3,10.1001/jama.2010.675,Viral etiology of severe pneumonia among Kenya...,CONTEXT: Pneumonia is the leading cause of chi...,2010.0,5.0,JAMA,,,,20501927.0,PMC2968755,3,2020-04-04 07:55:51.892454
4,10.1001/jama.2014.2116,Critically Ill Patients With Influenza A(H1N1)...,,2014.0,4.0,JAMA,,,,24566924.0,PMC6689404,4,2020-04-04 07:55:51.892454


In [159]:
# reorder the columns to match the SQL schema

df_datasource.columns

Index(['source', 'url', 'datasource_metadata_id'], dtype='object')

In [160]:
df_pub = df_pub[['pub_id', 'title', 'abstract', 'publication_year', 'publication_month', 'journal',
       'volume', 'issue', 'pages', 'doi', 'pmid', 'pmcid',
       'timestamp']]
df_who_metadata = df_who_metadata[['who_metadata_id', 'accession_number', 'ref', 'covidence', 'study', 'notes', 'tags',
       'pub_id']]
df_dimensions_metadata = df_dimensions_metadata[['dimensions_metadata_id', 'publication_id', 'source_uid', 'open_access',
       'publication_type', 'dimensions_url', 'mesh_terms', 'pub_id']]
df_cord_metadata = df_cord_metadata[[ 'cord19_metadata_id', 'source', 'license', 'full_text_file', 'ms_academic_id',
       'who_covidence', 'sha', 'full_text', 'pub_id']]
df_datasource = df_datasource[['datasource_metadata_id', 'source', 'url']]

### Dump to MySQL

Use this if you want to create a MySQL db.

In [ ]:
dtype_dict = {'pub_id':Integer, 'title':String, 'abstract':String, 'publication_year':Integer, 'publication_month':Integer, 'journal':String,
       'volume':String, 'issue':String, 'pages':String, 'doi':String, 'pmid':Integer, 'pmcid':String, 'timestamp':DateTime}

In [ ]:
# get API key
import configparser
config = configparser.ConfigParser()
config.read("credentials/conf.ini")
mysql_username = config["MYSQL"]["username"]
mysql_password = config["MYSQL"]["password"]
mysql_database = config["MYSQL"]["database"]

In [ ]:
sqlEngine = create_engine('mysql+pymysql://%s:%s@127.0.0.1/%s'%(mysql_username,mysql_password,mysql_database), pool_recycle=3600)
dbConnection = sqlEngine.connect()

In [ ]:
# main table
table_name = "pub"
try:
    frame = df_pub.to_sql(table_name, dbConnection, if_exists='append', index=False, index_label="pub_id", dtype=dtype_dict);
except ValueError as vx:
    print(vx)
except Exception as ex:   
    print(ex)
else:
    print("Table %s created successfully."%table_name);   
finally:
    dbConnection.close()

In [ ]:
sqlEngine = create_engine('mysql+pymysql://%s:%s@127.0.0.1/%s'%(mysql_username,mysql_password,mysql_database), pool_recycle=3600)
dbConnection = sqlEngine.connect()

In [ ]:
# other tables
try:
    frame = df_cord_metadata.to_sql("cord19_metadata", dbConnection, if_exists='append', index=True, index_label="cord19_metadata_id")
    frame = df_who_metadata.to_sql("who_metadata", dbConnection, if_exists='append', index=True, index_label="who_metadata_id")
    frame = df_dimensions_metadata.to_sql("dimensions_metadata", dbConnection, if_exists='append', index=True, index_label="dimensions_metadata_id")
    frame = df_datasource.to_sql("datasource", dbConnection, if_exists='append', index=True, index_label="datasource_id")
except ValueError as vx:
    print(vx)
except Exception as ex:   
    print(ex)
else:
    print("Tables created successfully.");   
finally:
    dbConnection.close()

In [ ]:
sqlEngine = create_engine('mysql+pymysql://%s:%s@127.0.0.1/%s'%(mysql_username,mysql_password,mysql_database), pool_recycle=3600)
dbConnection = sqlEngine.connect()

In [ ]:
# last table
try:
    frame = df_pub_to_datasource.to_sql("pub_datasource", dbConnection, if_exists='append', index=False, index_label=["pub_id","datasource_id"])
except ValueError as vx:
    print(vx)
except Exception as ex:   
    print(ex)
else:
    print("Table created successfully.");   
finally:
    dbConnection.close()

In [163]:
### Export the df_pub dataframe for further use

df_pub.to_csv("datasets_output/df_pub.csv", compression="gzip", index=False)

In [162]:
# export TSV for ingestion

df_pub.to_csv("datasets_output/sql_tables/pub.csv",index=False,sep="\t",header=False)
df_cord_metadata.to_csv("datasets_output/sql_tables/cord19_metadata.csv",index=False,sep="\t",header=False)
df_dimensions_metadata.to_csv("datasets_output/sql_tables/dimensions_metadata.csv",index=False,sep="\t",header=False)
df_who_metadata.to_csv("datasets_output/sql_tables/who_metadata.csv",index=False,sep="\t",header=False)
df_datasource.to_csv("datasets_output/sql_tables/datasource.csv",index=False,sep="\t",header=False)
df_pub_to_datasource.to_csv("datasets_output/sql_tables/pub_datasource.csv",index=False,sep="\t",header=False)